## Loading libraries

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM, Dense
from keras.models import Sequential
from PIL import Image
import matplotlib.pyplot as plt

## Mounting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Training data setup

### Unzipping folder

In [ ]:
!unzip -uq "/content/drive/My Drive/train.zip" -d "/content"

### Viewing content

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Shell_AI_train/train.csv')
print(df.head())

  DATE (MM/DD)    MST  ...  Moisture  Albedo (CMP11)
0          1/1  00:00  ...       0.0             0.0
1          1/1  00:01  ...       0.0             0.0
2          1/1  00:02  ...       0.0             0.0
3          1/1  00:03  ...       0.0             0.0
4          1/1  00:04  ...       0.0             0.0

[5 rows x 17 columns]


### Check number of values for each date

In [ ]:
print(df['DATE (MM/DD)'].value_counts())

10/15    1440
4/30     1440
7/10     1440
6/5      1440
11/30    1440
         ... 
7/25     1440
8/25     1440
5/18     1440
8/15     1440
7/1      1440
Name: DATE (MM/DD), Length: 366, dtype: int64


In [ ]:
print(df['MST'].value_counts())

00:04    366
19:37    366
07:17    366
19:40    366
03:43    366
        ... 
11:57    366
04:28    366
16:14    366
20:55    366
08:06    366
Name: MST, Length: 1440, dtype: int64


In [ ]:
selected_row = df[df['DATE (MM/DD)'] == '1/1']
selected_row = selected_row[selected_row['MST'] == '07:40']
print(selected_row)

    DATE (MM/DD)    MST  ...  Moisture  Albedo (CMP11)
460          1/1  07:40  ...       0.0          0.1883

[1 rows x 17 columns]


## VAE architecture and training

### VAE architecture

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
latent_dim = 10

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 262144)       0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 262144)            2883584   
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 256, 256, 32)      18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 256, 256, 3)       867       
Total params: 2,939,843
Trainable params: 2,939,843
Non-trainable params: 0
_________________________________________________

In [6]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Data preparation for VAE

In [ ]:
path = '/content/train'
files = os.listdir(path)
count_folder = 0
image_inputs = []
for f in files:
    #print(f)
    image_files = os.listdir(path+'/'+f)
    
    count_img = 0
    for img in image_files:
        #print(img)
        sample_image = Image.open(path+'/'+f+'/'+img)
        sample_image = sample_image.resize((256,256))
        sample_image = np.array(sample_image)
        image_inputs.append(sample_image)
        
        count_img += 1
        if count_img == 40:
            break
        
    count_folder += 1
    print('Folder: ',count_folder)
    if count_folder == 100:
        break
print(count_folder, count_img)
image_inputs = np.array(image_inputs).astype('float32') / 255.0
print(image_inputs.shape)


Folder:  1
Folder:  2
Folder:  3
Folder:  4
Folder:  5
Folder:  6
Folder:  7
Folder:  8
Folder:  9
Folder:  10
Folder:  11
Folder:  12
Folder:  13
Folder:  14
Folder:  15
Folder:  16
Folder:  17
Folder:  18
Folder:  19
Folder:  20
Folder:  21
Folder:  22
Folder:  23
Folder:  24
Folder:  25
Folder:  26
Folder:  27
Folder:  28
Folder:  29
Folder:  30
Folder:  31
Folder:  32
Folder:  33
Folder:  34
Folder:  35
Folder:  36
Folder:  37
Folder:  38
Folder:  39
Folder:  40
Folder:  41
Folder:  42
Folder:  43
Folder:  44
Folder:  45
Folder:  46
Folder:  47
Folder:  48
Folder:  49
Folder:  50
Folder:  51
Folder:  52
Folder:  53
Folder:  54
Folder:  55
Folder:  56
Folder:  57
Folder:  58
Folder:  59
Folder:  60
Folder:  61
Folder:  62
Folder:  63
Folder:  64
Folder:  65
Folder:  66
Folder:  67
Folder:  68
Folder:  69
Folder:  70
Folder:  71
Folder:  72
Folder:  73
Folder:  74
Folder:  75
Folder:  76
Folder:  77
Folder:  78
Folder:  79
Folder:  80
Folder:  81
Folder:  82
Folder:  83
Folder:  84
F

### Training VAE and saving weights

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(image_inputs, epochs=100, batch_size=64) 

In [ ]:
path='/content/drive/My Drive/Shell_AI_train/VAE_weights'
vae.save_weights(path, save_format='tf')

### Loading saved model

In [7]:
loaded_model = VAE(encoder, decoder)
savedModel = loaded_model.load_weights('/content/drive/My Drive/Shell_AI_train//VAE_weights')
print('Model Loaded!')

Model Loaded!


In [ ]:
image_filepaths = ['/content/train/0101/0101074000.jpg']
x_test = []
for img in image_filepaths:
    x_test.append(np.array(Image.open(img).resize((256,256))))
x_test = np.array(x_test).astype('float32') / 255.0
latent_vectors = loaded_model.encoder.predict(x_test)[0][0]
print(latent_vectors)

[-0.2699054  -0.04153442  0.0513358   0.10325338 -0.15685268  0.10583416
 -0.1099227   0.05887362 -0.58405864 -0.07685094]


In [ ]:
print(df.columns)

Index(['DATE (MM/DD)', 'MST', 'Global CMP22 (vent/cor) [W/m^2]',
       'Direct sNIP [W/m^2]', 'Azimuth Angle [degrees]',
       'Tower Dry Bulb Temp [deg C]', 'Tower Wet Bulb Temp [deg C]',
       'Tower Dew Point Temp [deg C]', 'Tower RH [%]', 'Total Cloud Cover [%]',
       'Peak Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]',
       'Station Pressure [mBar]', 'Precipitation (Accumulated) [mm]',
       'Snow Depth [cm]', 'Moisture', 'Albedo (CMP11)'],
      dtype='object')


### Converting -1 values to 0

In [ ]:
df['Total Cloud Cover [%]'].replace([-1],[0],inplace=True)

### Checking for null values

In [ ]:
df.isnull().sum() 

DATE (MM/DD)                             0
MST                                      0
Global CMP22 (vent/cor) [W/m^2]          0
Direct sNIP [W/m^2]                      0
Azimuth Angle [degrees]                  0
Tower Dry Bulb Temp [deg C]              0
Tower Wet Bulb Temp [deg C]              0
Tower Dew Point Temp [deg C]             0
Tower RH [%]                             0
Total Cloud Cover [%]                    0
Peak Wind Speed @ 6ft [m/s]              0
Avg Wind Direction @ 6ft [deg from N]    0
Station Pressure [mBar]                  0
Precipitation (Accumulated) [mm]         0
Snow Depth [cm]                          0
Moisture                                 0
Albedo (CMP11)                           0
dtype: int64

## LSTM architecture and training

### Creating data for LSTM training

In [ ]:
path = '/content/train'
files = os.listdir(path)
count_folder = 0
input_sequences_dict = {}
output_sequences_dict = {}
features_considered = ['Global CMP22 (vent/cor) [W/m^2]',
       'Direct sNIP [W/m^2]', 'Azimuth Angle [degrees]',
       'Tower Dry Bulb Temp [deg C]', 'Tower Wet Bulb Temp [deg C]',
       'Tower Dew Point Temp [deg C]', 'Tower RH [%]', 
       'Peak Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]',
       'Station Pressure [mBar]', 'Precipitation (Accumulated) [mm]',
       'Snow Depth [cm]', 'Moisture', 'Albedo (CMP11)', 'Total Cloud Cover [%]']

for f in files:
    #print(f)
    image_files = os.listdir(path+'/'+f)
    
    count_img = 0
    full_vectors = []
    output_vectors = []
    for img in image_files:
        #print(img)
        img_val = Image.open(path+'/'+f+'/'+img)
        img_val = img_val.resize((256,256))
        img_val = [np.array(img_val)]
        img_val = np.array(img_val).astype('float32') / 255.0
        latent_vectors = list(loaded_model.encoder.predict(img_val)[0][0])
        #print(latent_vectors)
        cur_date = str(int(f[:2])) + '/' + str(int(f[2:]))
        #print(cur_date)
        cur_time = img.split('.')[0]
        cur_time = cur_time[4:6] + ':' + cur_time[6:8]
        #print(cur_time)
        selected_row = df[df['DATE (MM/DD)'] == cur_date]
        selected_row = selected_row[selected_row['MST'] == cur_time]
        data_vectors = []
        try:
          for feat in features_considered:
            data_vectors.append(float(selected_row[feat]))
          full_vectors.append(latent_vectors + data_vectors)
          cur_out_vector = []
          output_vectors.append(float(selected_row['Total Cloud Cover [%]']))
        except:
          pass
        count_img += 1
        if count_img == 40:
            break
    input_sequences_dict[cur_date] = full_vectors
    output_sequences_dict[cur_date] = output_vectors

    count_folder += 1
    print('Folder: ', count_folder)
    #if count_folder == 2:
        #break

Folder:  1
Folder:  2
Folder:  3
Folder:  4
Folder:  5
Folder:  6
Folder:  7
Folder:  8
Folder:  9
Folder:  10
Folder:  11
Folder:  12
Folder:  13
Folder:  14
Folder:  15
Folder:  16
Folder:  17
Folder:  18
Folder:  19
Folder:  20
Folder:  21
Folder:  22
Folder:  23
Folder:  24
Folder:  25
Folder:  26
Folder:  27
Folder:  28
Folder:  29
Folder:  30
Folder:  31
Folder:  32
Folder:  33
Folder:  34
Folder:  35
Folder:  36
Folder:  37
Folder:  38
Folder:  39
Folder:  40
Folder:  41
Folder:  42
Folder:  43
Folder:  44
Folder:  45
Folder:  46
Folder:  47
Folder:  48
Folder:  49
Folder:  50
Folder:  51
Folder:  52
Folder:  53
Folder:  54
Folder:  55
Folder:  56
Folder:  57
Folder:  58
Folder:  59
Folder:  60
Folder:  61
Folder:  62
Folder:  63
Folder:  64
Folder:  65
Folder:  66
Folder:  67
Folder:  68
Folder:  69
Folder:  70
Folder:  71
Folder:  72
Folder:  73
Folder:  74
Folder:  75
Folder:  76
Folder:  77
Folder:  78
Folder:  79
Folder:  80
Folder:  81
Folder:  82
Folder:  83
Folder:  84
F

KeyboardInterrupt: ignored

In [ ]:
print(input_sequences_dict['3/31'])

[[-0.2294909, -0.0630684, 0.06996822, 0.074453436, -0.063876666, 0.04023607, -0.11971003, 0.046900786, -0.5512519, -0.08772144, 6.39404, -1.1869, 276.04363, 15.08, 4.695, -8.405, 17.43, 6.45, 273.5, 811.3960000000002, 0.0, 0.8340000000000001, 0.0, 0.0], [-0.26682532, -0.05661689, 0.008315764, 0.065918736, -0.14080785, 0.11042614, -0.068679035, 0.099684164, -0.57226366, -0.08756025, 936.102, 1026.96, 157.12689, 15.96, 6.138, -4.662, 22.78, 3.95, 38.23, 814.684, 0.0, 1.134, 0.0, 0.1778], [-0.24607198, -0.038348027, 0.066233784, 0.064619064, -0.09147489, 0.07045599, -0.09418832, 0.05147448, -0.5486262, -0.06040448, 156.14, 193.038, 259.76503, 15.87, 5.716, -6.084, 20.28, 1.45, 126.5, 811.7760000000002, 0.0, 0.8809999999999999, 0.0, 0.1842], [-0.20390595, -0.057251967, 0.0784675, 0.10021931, -0.07569167, 0.043227628, -0.09600215, 0.003867887, -0.50192606, -0.090959534, 8.33411, 6.98796, 85.81266, 5.416, -0.013, -7.612999999999999, 35.66, 3.2, 289.8, 817.185, 0.0, 0.983, 0.0, 0.0795], [-0.2

In [ ]:
print(output_sequences_dict['3/31'])

[0.0, 32.0, 18.0, 0.0, 31.0, 21.0, 10.0, 46.0, 21.0, 34.0, 61.0, 56.0, 57.0, 41.0, 35.0, 24.0, 59.0, 58.0, 7.0, 0.0, 33.0, 6.0, 63.0, 62.0, 29.0, 22.0, 37.0, 43.0, 4.0, 29.0, 0.0, 32.0, 32.0, 30.0, 36.0, 44.0, 54.0, 25.0, 27.0, 35.0]


### Code to prepare LSTM prediction sequences

In [ ]:
# split a univariate sequence into samples based on sequence dictionaries
def split_sequence_keys(input_sequence_dict, output_sequence_dict, n_steps):
  X, y = list(), list()
  for key in input_sequence_dict.keys():
   input_sequence = input_sequence_dict[key]
   output_sequence = output_sequence_dict[key]
   for j in range(len(input_sequence)):
		  # find the end of this pattern
		  end_ix = j + n_steps + n_steps
		  # check if we are beyond the sequence
		  if end_ix > len(input_sequence)-1:
			  break
		  # gather input and output parts of the pattern
		  seq_x, seq_y = input_sequence[j:end_ix-n_steps], output_sequence[end_ix-n_steps:end_ix]
		  X.append(seq_x)
		  y.append(seq_y)
  return np.array(X), np.array(y)

In [ ]:
# choose a number of time steps
n_steps = 12
n_input_features = 24
n_output_features = 12

In [ ]:
# split into samples
#X, y = split_sequence(raw_input_seq, raw_output_seq, n_steps)
X, y = split_sequence_keys(input_sequences_dict, output_sequences_dict, n_steps)

In [ ]:
# summarize the data
for i in range(len(X)):
	print(X[i],y[i])
	break

[[-3.13014567e-01  1.61993876e-02  4.44833525e-02  1.18205212e-01
  -1.67712480e-01  2.00000674e-01 -6.87756985e-02  7.94985369e-02
  -5.55178523e-01 -5.48711866e-02  9.38801000e+01  4.15421000e-01
   1.92904280e+02  2.08400000e+00 -1.77000000e+00 -7.57000000e+00
   4.52800000e+01  4.20000000e+00  1.56800000e+01  8.14709000e+02
   0.00000000e+00  5.04000000e-01  0.00000000e+00  1.75200000e-01]
 [-2.54202008e-01 -6.16296269e-02  6.33414090e-03  1.89165697e-02
  -1.26158595e-01  7.85578042e-02 -6.67587593e-02  9.29069221e-02
  -5.89182615e-01 -9.18417946e-02  5.55211000e+02  9.34274000e+02
   1.49411120e+02  2.01500000e+00 -1.30800000e+00 -5.90800000e+00
   5.25600000e+01  3.95000000e+00  5.43800000e+01  8.13216000e+02
   0.00000000e+00  5.61000000e-01  0.00000000e+00  2.17900000e-01]
 [-2.73119122e-01 -1.24791227e-02  4.36056629e-02  7.40202665e-02
  -9.60169360e-02  1.12747528e-01 -8.66775364e-02  6.93809763e-02
  -5.74246049e-01 -9.32789892e-02  1.25223000e+02  2.85158000e+01
   1.260

### LSTM architecture

In [ ]:
# define model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_input_features)))
lstm_model.add(Dense(n_output_features))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
print(X.shape)

(5588, 12, 24)


In [ ]:
print(y.shape)

(5588, 12)


In [ ]:
X = X.reshape((X.shape[0], X.shape[1], n_input_features))

### LSTM training and saving model



In [ ]:
# fit model
lstm_model.fit(X, y, epochs=100)

Epoch 1/100
175/175 [==============================] - 9s 41ms/step - loss: 354627.2812
Epoch 2/100
175/175 [==============================] - 7s 42ms/step - loss: 348566.3125
Epoch 3/100
175/175 [==============================] - 7s 41ms/step - loss: 347838.9062
Epoch 4/100
175/175 [==============================] - 7s 42ms/step - loss: 347054.2812
Epoch 5/100
175/175 [==============================] - 7s 42ms/step - loss: 346680.1875
Epoch 6/100
175/175 [==============================] - 7s 41ms/step - loss: 346167.7188
Epoch 7/100
175/175 [==============================] - 7s 42ms/step - loss: 345821.5938
Epoch 8/100
175/175 [==============================] - 7s 42ms/step - loss: 345537.8438
Epoch 9/100
175/175 [==============================] - 7s 42ms/step - loss: 345435.4688
Epoch 10/100
175/175 [==============================] - 7s 41ms/step - loss: 346046.4688
Epoch 11/100
175/175 [==============================] - 7s 41ms/step - loss: 345419.4375
Epoch 12/100
175/175 [========

In [ ]:
lstm_model.save('/content/drive/My Drive/Shell_AI_train/lstm_model_120min.h5')
#Choose between lstm_model_30min.h5, lstm_model_60min.h5, lstm_model_90min.h5, lstm_model_120min.h5

### LSTM model loading

In [ ]:
loaded_lstm_model = keras.models.load_model('/content/drive/My Drive/Shell_AI_train/lstm_model_15min.h5')

In [ ]:
x_input = np.array(
    [[-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  1.06589000e+03,  9.56709000e+02,
   1.34165270e+02,  2.93500000e+01,  1.50420000e+01,  5.77200000e+00,
   2.25200000e+01,  3.20000000e+00,  2.50400000e+01,  8.17512000e+02,
   0.00000000e+00,  1.08800000e+00,  0.00000000e+00,  1.62000000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  5.14860000e+02,  8.64036000e+02,
   8.54014800e+01,  2.32800000e+01,  1.16670000e+01,  2.66700000e+00,
   2.59000000e+01,  2.70000000e+00,  2.89300000e+02,  8.18295000e+02,
   0.00000000e+00,  7.59000000e-01,  0.00000000e+00,  2.09800000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  3.34885000e+02,  7.35075000e+02,
   7.66483800e+01,  2.12100000e+01,  1.09820000e+01,  2.80200000e+00,
   2.96600000e+01,  0.00000000e+00,  0.00000000e+00,  8.18052000e+02,
   0.00000000e+00,  7.44000000e-01,  0.00000000e+00,  2.32800000e-01],
     [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  1.06589000e+03,  9.56709000e+02,
   1.34165270e+02,  2.93500000e+01,  1.50420000e+01,  5.77200000e+00,
   2.25200000e+01,  3.20000000e+00,  2.50400000e+01,  8.17512000e+02,
   0.00000000e+00,  1.08800000e+00,  0.00000000e+00,  1.62000000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  5.14860000e+02,  8.64036000e+02,
   8.54014800e+01,  2.32800000e+01,  1.16670000e+01,  2.66700000e+00,
   2.59000000e+01,  2.70000000e+00,  2.89300000e+02,  8.18295000e+02,
   0.00000000e+00,  7.59000000e-01,  0.00000000e+00,  2.09800000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  3.34885000e+02,  7.35075000e+02,
   7.66483800e+01,  2.12100000e+01,  1.09820000e+01,  2.80200000e+00,
   2.96600000e+01,  0.00000000e+00,  0.00000000e+00,  8.18052000e+02,
   0.00000000e+00,  7.44000000e-01,  0.00000000e+00,  2.32800000e-01],
     [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  1.06589000e+03,  9.56709000e+02,
   1.34165270e+02,  2.93500000e+01,  1.50420000e+01,  5.77200000e+00,
   2.25200000e+01,  3.20000000e+00,  2.50400000e+01,  8.17512000e+02,
   0.00000000e+00,  1.08800000e+00,  0.00000000e+00,  1.62000000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  5.14860000e+02,  8.64036000e+02,
   8.54014800e+01,  2.32800000e+01,  1.16670000e+01,  2.66700000e+00,
   2.59000000e+01,  2.70000000e+00,  2.89300000e+02,  8.18295000e+02,
   0.00000000e+00,  7.59000000e-01,  0.00000000e+00,  2.09800000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  3.34885000e+02,  7.35075000e+02,
   7.66483800e+01,  2.12100000e+01,  1.09820000e+01,  2.80200000e+00,
   2.96600000e+01,  0.00000000e+00,  0.00000000e+00,  8.18052000e+02,
   0.00000000e+00,  7.44000000e-01,  0.00000000e+00,  2.32800000e-01],
   [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  1.06589000e+03,  9.56709000e+02,
   1.34165270e+02,  2.93500000e+01,  1.50420000e+01,  5.77200000e+00,
   2.25200000e+01,  3.20000000e+00,  2.50400000e+01,  8.17512000e+02,
   0.00000000e+00,  1.08800000e+00,  0.00000000e+00,  1.62000000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  5.14860000e+02,  8.64036000e+02,
   8.54014800e+01,  2.32800000e+01,  1.16670000e+01,  2.66700000e+00,
   2.59000000e+01,  2.70000000e+00,  2.89300000e+02,  8.18295000e+02,
   0.00000000e+00,  7.59000000e-01,  0.00000000e+00,  2.09800000e-01],
 [-3.99366161e-03,  6.03996217e-02,  6.81698024e-02, -5.42859808e-02,
   1.83254704e-02, -6.66188300e-02,  4.71722931e-02,  3.89583744e-02,
  -1.89637132e-02, -1.38268266e-02,  3.34885000e+02,  7.35075000e+02,
   7.66483800e+01,  2.12100000e+01,  1.09820000e+01,  2.80200000e+00,
   2.96600000e+01,  0.00000000e+00,  0.00000000e+00,  8.18052000e+02,
   0.00000000e+00,  7.44000000e-01,  0.00000000e+00,  2.32800000e-01]]
)
x_input = x_input.reshape((1, n_steps, n_input_features))
yhat = loaded_lstm_model.predict(x_input, verbose=0)
print(yhat)

[[-43.44518     13.861025    24.397915     0.66326135  20.0931
    7.3613195  -12.346386   -26.01548    -88.20771    -70.21154
  -80.27909    -63.76428   ]]


## Test set preparation

### Unzip test folder

In [8]:
!unzip -uq "/content/drive/My Drive/test.zip" -d "/content"

### Sequence preparation

In [37]:
path = '/content/test'
files = os.listdir(path)
#print(files)
input_sequences_dict = {}
output_sequences_dict = {}
features_considered = ['Global CMP22 (vent/cor) [W/m^2]',
       'Direct sNIP [W/m^2]', 'Azimuth Angle [degrees]',
       'Tower Dry Bulb Temp [deg C]', 'Tower Wet Bulb Temp [deg C]',
       'Tower Dew Point Temp [deg C]', 'Tower RH [%]', 
       'Peak Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]',
       'Station Pressure [mBar]', 'Precipitation (Accumulated) [mm]',
       'Snow Depth [cm]', 'Moisture', 'Albedo (CMP11)']
#output_df = {'Scenario':[], '30_min_horizon':[] , '60_min_horizon':[], '90_min_horizon':[], '120_min_horizon':[]}

In [38]:
horizon_dict_30_min = {}
horizon_dict_60_min = {}
horizon_dict_90_min = {}
horizon_dict_120_min = {}

In [48]:
loaded_lstm_model = keras.models.load_model('/content/drive/My Drive/Shell_AI_train/lstm_model_30min.h5') 
#Choose between lstm_model_30min.h5, lstm_model_60min.h5, lstm_model_90min.h5, lstm_model_120min.h5
n_steps = 3 
#Choose between 3, 6, 9, 12
n_input_features = 24

### Perform predictions

In [49]:
count_folder = 0
for f in files:
    data_file = 'weather_data.csv'

    image_files = []
    for filename in glob.glob(path + '/' + f + '/*.jpg'): 
      filename = filename.split('/')[-1]
      filename = filename.split('.')[0]
      image_files.append(int(filename))
    image_files = sorted(image_files)[-n_steps:]
    #print(image_files[-3:])

    cur_df = pd.read_csv(path + '/' + f + '/' + data_file)
    #print(cur_df.head())

    count_img = 0
    full_vectors = []
    try:
      for img in image_files:
        #print(img)
        img = str(img) + '.jpg'
        img_val = Image.open(path+'/'+f+'/'+img)
        img_val = img_val.resize((256,256))
        img_val = [np.array(img_val)]
        img_val = np.array(img_val).astype('float32') / 255.0
        latent_vectors = list(loaded_model.encoder.predict(img_val)[0][0])
        scenario_number = int(img.split('.')[0])
        selected_row = cur_df[cur_df['Time [Mins]'] == scenario_number]
        #print(selected_row)
        data_vectors = []
        for feat in features_considered:
            data_vectors.append(float(selected_row[feat]))
        full_vectors.append(latent_vectors + data_vectors)
        #print(count_img)
        count_img += 1

    
      #print(full_vectors)
      full_vectors = np.array(full_vectors)
      full_vectors = full_vectors.reshape((1, n_steps, n_input_features))
      yhat = loaded_lstm_model.predict(full_vectors, verbose=0)[0][-1]
      print(yhat)
      if yhat < 0:
        yhat = 0
      if yhat > 100:
        yhat = 100
      #output_df['Scenario'].append(int(f))
      #output_df['30_min_horizon'].append(yhat)
      horizon_dict_30_min[int(f)] = yhat
    except:
      pass
    count_folder += 1
    print(count_folder)

-46.407024
1
2.925826
2
20.789171
3
-92.952675
4
89.28518
5
22.205824
6
-26.858007
7
-18.299726
8
-157.93013
9
55.988857
10
124.73139
11
103.29031
12
64.55965
13
-14.192694
14
-139.1988
15
-7.1202574
16
-72.303375
17
-7.383618
18
1.596409
19
-59.256355
20
14.158861
21
59.020412
22
-52.546406
23
-98.326416
24
-24.255493
25
68.31713
26
-64.461075
27
-14.036546
28
41.185963
29
-86.22634
30
-17.899187
31
61.312218
32
-24.677698
33
-96.050186
34
51.434784
35
-75.04739
36
1.4932021
37
15.832121
38
22.994545
39
105.271996
40
64.43258
41
-61.423225
42
21.927273
43
33.998383
44
-132.02698
45
-21.991682
46
70.11186
47
10.223568
48
-21.853807
49
-57.15285
50
5.583024
51
-10.855954
52
12.933608
53
-68.489
54
-91.71973
55
17.047266
56
14.036635
57
-28.11735
58
25.528717
59
87.36391
60
-5.458382
61
55.31151
62
-88.373085
63
-2.7278523
64
7.49084
65
100.23862
66
42.95934
67
-54.6761
68
-50.132553
69
59.1893
70
-36.024162
71
-17.250324
72
17.808048
73
-3.7096086
74
102.50456
75
37.345768
76
-19.766453

In [50]:
print(horizon_dict_30_min)

{71: 0, 45: 2.925826, 234: 20.789171, 225: 0, 75: 89.28518, 47: 22.205824, 54: 0, 209: 0, 51: 0, 244: 55.988857, 149: 100, 217: 100, 152: 64.55965, 115: 0, 133: 0, 91: 0, 142: 0, 271: 0, 53: 1.596409, 163: 0, 79: 14.158861, 3: 59.020412, 84: 0, 171: 0, 73: 0, 159: 68.31713, 137: 0, 99: 0, 17: 41.185963, 247: 0, 121: 0, 240: 61.312218, 207: 0, 264: 0, 87: 51.434784, 111: 0, 298: 1.4932021, 222: 15.832121, 226: 22.994545, 129: 100, 224: 64.43258, 135: 0, 22: 21.927273, 220: 33.998383, 118: 0, 243: 0, 107: 70.11186, 232: 10.223568, 153: 0, 56: 0, 130: 5.583024, 257: 0, 127: 12.933608, 5: 0, 221: 0, 164: 17.047266, 13: 14.036635, 211: 0, 63: 25.528717, 97: 87.36391, 119: 0, 194: 55.31151, 124: 0, 262: 0, 245: 7.49084, 239: 100, 55: 42.95934, 276: 0, 136: 0, 11: 59.1893, 146: 0, 147: 0, 289: 17.808048, 297: 0, 255: 100, 123: 37.345768, 158: 0, 261: 9.527512, 267: 0, 128: 0, 37: 0, 235: 0, 286: 49.855667, 77: 58.634327, 201: 13.449441, 213: 38.40657, 82: 0, 105: 83.47573, 275: 0, 85: 0, 68: 

### Turn into dataframe format

In [51]:
output_df = {'Scenario':[], '30_min_horizon':[] , '60_min_horizon':[], '90_min_horizon':[], '120_min_horizon':[]}
for i in range(1, 301):
  output_df['Scenario'].append(i)
  output_df['30_min_horizon'].append(horizon_dict_30_min[i])
  output_df['60_min_horizon'].append(horizon_dict_60_min[i])
  output_df['90_min_horizon'].append(horizon_dict_90_min[i])
  output_df['120_min_horizon'].append(horizon_dict_90_min[i])


In [52]:
print(output_df)
output_df = pd.DataFrame(output_df)
output_df = output_df.sort_values('Scenario')
output_df.head()

{'Scenario': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

,Scenario,30_min_horizon,60_min_horizon,90_min_horizon,120_min_horizon
0,1,67.754700,77.784813,49.207157,49.207157
1,2,47.627579,26.966404,0.000000,0.000000
2,3,59.020412,42.238464,38.526131,38.526131
3,4,29.425442,27.474157,24.754330,24.754330
4,5,0.000000,0.000000,0.000000,0.000000


### Prepare submission .csv file

In [53]:
from google.colab import files
output_df.to_csv('submission_cloudcover.csv', index=False)
files.download('submission_cloudcover.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>